In [128]:
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
import numpy as np

In [129]:
import tensorflow as tf
tf.compat.v1.get_default_graph()

In [130]:
data = pd.read_csv("kddcup99_csv.csv")

In [131]:
labels = ['normal.', 'buffer_overflow.', 'loadmodule.', 'perl.', 'neptune.', 'smurf.',
 'guess_passwd.', 'pod.', 'teardrop.', 'portsweep.', 'ipsweep.', 'land.', 'ftp_write.',
 'back.', 'imap.', 'satan.', 'phf.', 'nmap.', 'multihop.', 'warezmaster.', 'warezclient.',
 'spy.', 'rootkit.']

In [132]:
dos = ['Neptune', 'Smurf', 'Pod', 'Teardrop','land' ,'back' ,'mailbomb']

In [133]:
data.head(5)

,duration,protocol_type,service,...,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,...,0.0,0.0,normal
1,0,tcp,http,...,0.0,0.0,normal
2,0,tcp,http,...,0.0,0.0,normal
3,0,tcp,http,...,0.0,0.0,normal
4,0,tcp,http,...,0.0,0.0,normal


In [134]:
data.info

<bound method DataFrame.info of         duration protocol_type service  ... dst_host_rerror_rate  \
0              0           tcp    http  ...                  0.0   
1              0           tcp    http  ...                  0.0   
...          ...           ...     ...  ...                  ...   
494018         0           tcp    http  ...                  0.0   
494019         0           tcp    http  ...                  0.0   

        dst_host_srv_rerror_rate   label  
0                            0.0  normal  
1                            0.0  normal  
...                          ...     ...  
494018                       0.0  normal  
494019                       0.0  normal  

[494020 rows x 42 columns]>

In [135]:
list(data.columns)

['duration',
 'protocol_type',
 'service',
 'flag',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'lnum_compromised',
 'lroot_shell',
 'lsu_attempted',
 'lnum_root',
 'lnum_file_creations',
 'lnum_shells',
 'lnum_access_files',
 'lnum_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'label']

In [136]:
df.dropna(inplace=True,axis=1)
df

,0,1,2,...,39,40,41
0,0,tcp,http,...,0.0,0.0,normal.
1,0,tcp,http,...,0.0,0.0,normal.
...,...,...,...,...,...,...,...
494019,0,tcp,http,...,0.0,0.0,normal.
494020,0,tcp,http,...,0.0,0.0,normal.


In [137]:
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result))
        
def analyze(df):
    print()
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        unique_count = len(uniques)
        if unique_count>100:
            print("** {}:{} ({}%)".format(col,unique_count,int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])

In [138]:
analyze(df)


494021 rows
** 0:2495 (0%)
** 1:[icmp:57.41%,tcp:38.47%,udp:4.12%]
** 2:[ecr_i:56.96%,private:22.45%,http:13.01%,smtp:1.97%,other:1.46%,domain_u:1.19%,ftp_data:0.96%,eco_i:0.33%,ftp:0.16%,finger:0.14%,urp_i:0.11%,telnet:0.1%,ntp_u:0.08%,auth:0.07%,pop_3:0.04%,time:0.03%,csnet_ns:0.03%,remote_job:0.02%,imap4:0.02%,gopher:0.02%,domain:0.02%,discard:0.02%,iso_tsap:0.02%,systat:0.02%,echo:0.02%,shell:0.02%,rje:0.02%,whois:0.02%,sql_net:0.02%,printer:0.02%,nntp:0.02%,courier:0.02%,sunrpc:0.02%,netbios_ssn:0.02%,mtp:0.02%,bgp:0.02%,vmnet:0.02%,uucp:0.02%,klogin:0.02%,uucp_path:0.02%,ssh:0.02%,supdup:0.02%,nnsp:0.02%,login:0.02%,hostnames:0.02%,efs:0.02%,daytime:0.02%,netbios_ns:0.02%,link:0.02%,ldap:0.02%,pop_2:0.02%,netbios_dgm:0.02%,exec:0.02%,http_443:0.02%,name:0.02%,kshell:0.02%,ctf:0.02%,netstat:0.02%,Z39_50:0.02%,IRC:0.01%,urh_i:0.0%,X11:0.0%,tim_i:0.0%,red_i:0.0%,tftp_u:0.0%,pm_dump:0.0%]
** 3:[SF:76.6%,S0:17.61%,REJ:5.44%,RSTR:0.18%,RSTO:0.12%,SH:0.02%,S1:0.01%,S2:0.0%,RSTOS0:0.0%,

In [139]:
import pandas as pd
from tensorflow.keras.utils import get_file

pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 5)
df = pd.read_csv(path, header=None)
df.dropna(inplace=True,axis=1) 
df.columns = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'outcome'
]

# hiển thị 5 hàng
pd.set_option('display.max_columns', 5)
pd.set_option('display.max_rows', 5)
df

,duration,protocol_type,...,dst_host_srv_rerror_rate,outcome
0,0,tcp,...,0.0,normal.
1,0,tcp,...,0.0,normal.
...,...,...,...,...,...
494019,0,tcp,...,0.0,normal.
494020,0,tcp,...,0.0,normal.


In [140]:
outcome = df.outcome

In [141]:
normal_mask = df['outcome']=='normal.'
attack_mask = df['outcome']!='normal.'

df.drop('outcome',axis=1,inplace=True)

df_normal = df[normal_mask]
df_attack = df[attack_mask]
#phân loại 
print(f"so lan binh thuong: {len(df_normal)}")
print(f"so lan bi tan cong: {len(df_attack)}")

so lan binh thuong: 97278
so lan bi tan cong: 396743


In [142]:
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd
    
#Mã hóa các giá trị văn bản thành các biến giả (tức là [1,0,0], [0,1,0], [0,0,1] cho màu đỏ, xanh lục, xanh lam)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

In [143]:
pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 5)

encode_numeric_zscore(df, 'duration')
encode_text_dummy(df, 'protocol_type')
encode_text_dummy(df, 'service')
encode_text_dummy(df, 'flag')
encode_numeric_zscore(df, 'src_bytes')
encode_numeric_zscore(df, 'dst_bytes')
encode_text_dummy(df, 'land')
encode_numeric_zscore(df, 'wrong_fragment')
encode_numeric_zscore(df, 'urgent')
encode_numeric_zscore(df, 'hot')
encode_numeric_zscore(df, 'num_failed_logins')
encode_text_dummy(df, 'logged_in')
encode_numeric_zscore(df, 'num_compromised')
encode_numeric_zscore(df, 'root_shell')
encode_numeric_zscore(df, 'su_attempted')
encode_numeric_zscore(df, 'num_root')
encode_numeric_zscore(df, 'num_file_creations')
encode_numeric_zscore(df, 'num_shells')
encode_numeric_zscore(df, 'num_access_files')
encode_numeric_zscore(df, 'num_outbound_cmds')
encode_text_dummy(df, 'is_host_login')
encode_text_dummy(df, 'is_guest_login')
encode_numeric_zscore(df, 'count')
encode_numeric_zscore(df, 'srv_count')
encode_numeric_zscore(df, 'serror_rate')
encode_numeric_zscore(df, 'srv_serror_rate')
encode_numeric_zscore(df, 'rerror_rate')
encode_numeric_zscore(df, 'srv_rerror_rate')
encode_numeric_zscore(df, 'same_srv_rate')
encode_numeric_zscore(df, 'diff_srv_rate')
encode_numeric_zscore(df, 'srv_diff_host_rate')
encode_numeric_zscore(df, 'dst_host_count')
encode_numeric_zscore(df, 'dst_host_srv_count')
encode_numeric_zscore(df, 'dst_host_same_srv_rate')
encode_numeric_zscore(df, 'dst_host_diff_srv_rate')
encode_numeric_zscore(df, 'dst_host_same_src_port_rate')
encode_numeric_zscore(df, 'dst_host_srv_diff_host_rate')
encode_numeric_zscore(df, 'dst_host_serror_rate')
encode_numeric_zscore(df, 'dst_host_srv_serror_rate')
encode_numeric_zscore(df, 'dst_host_rerror_rate')
encode_numeric_zscore(df, 'dst_host_srv_rerror_rate')

# hiển thị 5 hàng

In [144]:
df.dropna(inplace=True,axis=1)
df[0:5]

,duration,src_bytes,dst_bytes,...,is_host_login-0,is_guest_login-0,is_guest_login-1
0,-0.067792,-0.002879,0.138664,...,1,1,0
1,-0.067792,-0.002820,-0.011578,...,1,1,0
2,-0.067792,-0.002824,0.014179,...,1,1,0
3,-0.067792,-0.002840,0.014179,...,1,1,0
4,-0.067792,-0.002842,0.035214,...,1,1,0


In [145]:
df.columns.to_list()

['duration',
 'src_bytes',
 'dst_bytes',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'protocol_type-icmp',
 'protocol_type-tcp',
 'protocol_type-udp',
 'service-IRC',
 'service-X11',
 'service-Z39_50',
 'service-auth',
 'service-bgp',
 'service-courier',
 'service-csnet_ns',
 'service-ctf',
 'service-daytime',
 'service-discard',
 'service-domain',
 'service-domain_u',
 'service-echo',
 'service-eco_i',
 'service-ecr_i',
 'service-e

In [146]:
df.shape

(494021, 120)

In [147]:
outcome.shape

(494021,)

In [148]:
# X = df.drop(['outcome'], axis=1)
X = np.asarray(df).astype(np.float32)

In [149]:
Y = outcome.apply(lambda x: labels.index(x))

In [150]:
Y.shape

(494021,)

In [151]:
Y[:10]

0    0
1    0
    ..
8    0
9    0
Name: outcome, Length: 10, dtype: int64

In [152]:
type(Y)

pandas.core.series.Series

In [153]:
X[:10]

array([[-0.06779165, -0.00287852,  0.13866426, ...,  1.        ,
         1.        ,  0.        ],
       [-0.06779165, -0.00281983, -0.01157786, ...,  1.        ,
         1.        ,  0.        ],
       [-0.06779165, -0.00282388,  0.0141788 , ...,  1.        ,
         1.        ,  0.        ],
       ...,
       [-0.06779165, -0.00290079,  0.09741125, ...,  1.        ,
         1.        ,  0.        ],
       [-0.06779165, -0.00284918, -0.02171708, ...,  1.        ,
         1.        ,  0.        ],
       [-0.06779165, -0.00284716, -0.00249795, ...,  1.        ,
         1.        ,  0.        ]], dtype=float32)

In [154]:
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X, Y, test_size=0.3)

In [155]:
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

In [156]:
print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

(345814, 120) (74103, 120) (74104, 120) (345814,) (74103,) (74104,)


In [157]:
model = Sequential([
                        Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(120,)),
                        Dropout(0.3),
                        Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
                        Dropout(0.3),
                        Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
                        Dropout(0.3),
                        Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
                        Dropout(0.2),
                        Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
                        Dropout(0.3),
                        Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)),
                        Dropout(0.1),
                        Dense(23, activation='softmax')
                    ])

In [158]:
opt = Adam(lr=0.01)
# opt = SGD(lr=0.09, momentum=0.9)
model.compile(
                    optimizer=opt,
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

In [159]:
# hist = model.fit(X_train, Y_train, batch_size=32, epochs=100, validation_data=(X_val, Y_val))
# hist = model.fit(X_train, Y_train, batch_size=32, epochs=100, validation_data=(X_val, Y_val))
# model.save_weights("model.h5")

In [160]:
model.load_weights("model.h5")

In [161]:
prediction = model.predict(np.array(X_test))

In [162]:
np.array(X_test)

array([[-0.06779165, -0.00201738, -0.0262873 , ...,  1.        ,
         1.        ,  0.        ],
       [-0.06779165, -0.00306168, -0.0262873 , ...,  1.        ,
         1.        ,  0.        ],
       [-0.06779165, -0.00201738, -0.0262873 , ...,  1.        ,
         1.        ,  0.        ],
       ...,
       [-0.06779165, -0.00281174,  0.15064974, ...,  1.        ,
         1.        ,  0.        ],
       [-0.06779165, -0.00201738, -0.0262873 , ...,  1.        ,
         1.        ,  0.        ],
       [-0.06779165, -0.00306168, -0.0262873 , ...,  1.        ,
         1.        ,  0.        ]], dtype=float32)

In [163]:
print(prediction.shape)

(74104, 23)


In [164]:
def decode_onehot(arr):
    lenght = len(arr)
    max_index = 0;
    for i in range(0,lenght):
        if arr[i] > arr[max_index]:
            max = i
    return max_index

In [165]:
print(prediction[1])
print(decode_onehot(prediction[1]))

[2.0508535e-01 4.6299145e-05 2.3073133e-06 3.3795735e-07 2.0278092e-01
 5.7526827e-01 7.6286058e-05 4.7851115e-04 1.6336147e-03 2.0447993e-03
 2.5122815e-03 1.6925907e-05 3.4169741e-06 5.0806818e-03 2.2471180e-05
 2.7939898e-03 4.7256520e-08 3.7353410e-04 2.9273849e-06 2.4619207e-05
 1.7454670e-03 4.2111168e-07 6.4833871e-06]
0


In [166]:
Y_predict = np.empty(prediction.shape[0])
for i in range(0,prediction.shape[0]):
    Y_predict[i] = decode_onehot(prediction[i])

In [167]:
print(Y_test.tolist())

[5, 4, 5, 5, 5, 4, 5, 5, 5, 0, 0, 5, 4, 8, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 4, 5, 0, 5, 5, 5, 5, 5, 5, 5, 0, 5, 4, 4, 5, 5, 5, 0, 5, 0, 5, 0, 0, 0, 5, 5, 4, 5, 5, 4, 4, 5, 4, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 0, 4, 5, 5, 5, 5, 10, 5, 5, 0, 0, 5, 4, 5, 5, 5, 5, 0, 0, 4, 5, 5, 4, 5, 5, 0, 0, 0, 0, 4, 4, 4, 4, 5, 5, 5, 4, 5, 4, 4, 4, 5, 0, 0, 5, 0, 5, 0, 4, 5, 5, 5, 0, 5, 4, 5, 0, 4, 5, 5, 4, 4, 4, 5, 4, 5, 5, 5, 5, 5, 5, 0, 5, 0, 5, 5, 5, 4, 4, 5, 0, 5, 5, 4, 5, 0, 5, 5, 5, 5, 5, 5, 4, 5, 5, 13, 0, 4, 5, 5, 5, 5, 5, 4, 5, 5, 4, 0, 5, 5, 5, 4, 5, 5, 9, 0, 5, 5, 5, 5, 5, 5, 4, 5, 4, 0, 0, 4, 4, 5, 0, 0, 5, 4, 5, 5, 5, 5, 4, 5, 4, 4, 5, 4, 0, 5, 5, 5, 5, 0, 5, 4, 5, 4, 5, 5, 4, 5, 10, 5, 4, 5, 0, 0, 4, 5, 5, 5, 4, 4, 0, 5, 5, 0, 4, 5, 5, 5, 4, 0, 5, 4, 0, 4, 4, 5, 5, 5, 5, 0, 4, 5, 5, 5, 4, 4, 0, 5, 5, 0, 5, 5, 4, 5, 0, 4, 0, 5, 5, 4, 5, 4, 5, 4, 0, 4, 5, 0, 10, 5, 5, 5, 4, 5, 5, 5, 4, 0, 5, 5, 4, 4, 5, 5, 0, 5, 5, 5, 0, 4, 0, 4, 5, 20, 5, 5, 0, 5, 5, 0, 0, 5, 4, 5, 0

In [168]:
model.evaluate(X_test, Y_test)[1]

2316/2316 [==============================] - 3s 1ms/step - loss: 1.0790 - accuracy: 0.5687


0.5687277317047119

In [169]:
labels = ['normal.', 'buffer_overflow.', 'loadmodule.', 'perl.', 'neptune.', 'smurf.',
 'guess_passwd.', 'pod.', 'teardrop.', 'portsweep.', 'ipsweep.', 'land.', 'ftp_write.',
 'back.', 'imap.', 'satan.', 'phf.', 'nmap.', 'multihop.', 'warezmaster.', 'warezclient.',
 'spy.', 'rootkit.']
dos = ['Neptune.', 'Smurf.', 'Pod.', 'Teardrop.','land.' ,'back.' ,'mailbomb.']
num = len(prediction)
num_dos = 0
# print(Y_test.tolist())
print(Y_predict.tolist())
# for y in Y_predict:
#     print(labels[int(y)])
#     if(labels[int(y)] in dos):
#         num_dos += 1
# print(num_dos)
# print(num)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,